In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import sys
sys.path.append("libraries")
import evalfuns as ef
import numpy as np
import pandas as pd
import numpysimfunsNONuniform as sf
import distcomputing as dists
import pickle


In [ ]:
num_alts = 4
measures = list([f"KS{num_alts}", "fDP{num_alts}", "fCS{num_alts}"])
dMatDict = dists.genDistMatrices(4)
num2pyrank = dict(zip(range(13), dMatDict[f"KS{num_alts}"].columns))

In [ ]:
for dMat in dMatDict:
    istruelist = []
    for reliability in range(51,100,2):
        R = dists.weakRankings(numAlts)

        intSpace = np.arange(R.shape[0])
        markerDict = dict(zip(["KS","DP","CS"],["s","D","v"]))
        colorDict=dict(zip(["KS","DP","CS"], ["#C55B", "#55CB","#EC7B"]))
        if reliability != 51: oldCliqueDist = maxCliqueDist
        else: oldCliqueDist = None
        maxCliqueDist = min([int(1/(2*(1-reliability/100))), 6])

        if oldCliqueDist != maxCliqueDist:
            Cliquelist = []
            G=nx.Graph()
            plt.figure(figsize=(18,18))
            EDGES = []
            for w in range(1,maxCliqueDist+1):
                D2 = (dMatDict[f"{dMat}"] == w).values
                edges= set()
                for i in zip(R[(intSpace.reshape(1,-1) * D2).reshape(-1)], R[(intSpace.reshape(-1,1) * D2).reshape(-1)]):
                    try: 
                        if nx.shortest_path_length(G, *i, weight="weight")>=w: 
                            if frozenset(i) not in edges: 
                                G.add_edges_from([i], weight=w)
                                edges.add(frozenset(i))
                    except (nx.NodeNotFound, nx.NetworkXNoPath):  
                        if frozenset(i) not in edges: 
                            edges.add(frozenset(i))
                            G.add_edges_from([i], weight=w)
                edges = [tuple(i) for i in edges if len(i)==2]
                EDGES.append(edges)
            isolatedNodes = tuple(nx.isolates(G))
            G.remove_nodes_from(isolatedNodes)
            for w,edges in enumerate(EDGES):
                edges = tuple(i for i in edges if not i[0] in isolatedNodes and not i[1] in isolatedNodes)
                pos = nx.kamada_kawai_layout(G)
                nx.draw_networkx_edges(G, pos, edgelist=edges, alpha=.5, width = 10//(1+w))
            nx.draw_networkx_nodes(G, pos, node_size=1000, node_color=colorDict[dMat[:-1]][:-1])
            nx.draw_networkx_labels(G,pos, text_size=8)
            print()
            plt.show()
            Cliques = list(nx.find_cliques(G))
            Cliquelist = [set(i) for i in Cliques]
            #plt.savefig("Figures/"+ f"{dMat}{numAlts} Graph", bbox_inches='tight')
            print(f"{dMat} has the maximum clique-size of",max([len(i) for i in Cliques]), f"when clique-distances of up to {maxCliqueDist} are tolerated.")
            print(f"That means at most {max([len(i) for i in nx.find_cliques(G)])} clusters for bias lower than {1-1/(2*maxCliqueDist)}")
    print(istruelist)